# ĐỀ BÀI
<br> Có 𝑁 lớp 1,2, … , 𝑁 cần được xếp thời khóa biểu. Mỗi lớp 𝑖 có 𝑡(𝑖) là số tiết và 𝑔(𝑖) là giáo viên đã được phân công dạy lớp đó và 𝑠(𝑖) là số sinh viên của lớp. Có 𝑀 phòng học 1, 2, … , 𝑀, trong đó 𝑐(𝑖) là số chỗ ngồi của phòng 𝑖. Trong tuần có 5 ngày (từ thứ 2 đến thứ 5), mỗi ngày chia thành 12 tiết (6 tiết sáng và 6 tiết chiều). Các tiết của các ngày được đánh số lần lượt từ 1 đến 60.

<br> Hãy lập thời khóa biểu (xác định ngày, tiết và phòng gán cho mỗi lớp):
- Hai lớp có chung giáo viên thì phải xếp thời khóa biểu tách rời nhau 
- Số sinh viên trong mỗi lớp phải nhỏ hơn hoặc bằng số chỗ ngồi của phòng học
- Một môn học chỉ trong một buổi (sáng hoặc chiều một ngày nào đó)

<br> Mục tiêu: Số lớp được xếp thời khóa biểu là lớn nhất

# INPUT
<br> Line 1: ghi 𝑁 và 𝑀 (1 ≤ 𝑁 ≤ 1000, 1 ≤ 𝑀 ≤ 100)
<br> Line 𝑖 + 1 (𝑖 = 1, … , 𝑁): ghi 𝑡(𝑖), 𝑔(𝑖) và 𝑠(𝑖) (1 ≤ 𝑡(𝑖) ≤ 4, 1 ≤ 𝑔(𝑖) ≤ 100, 1 ≤ 𝑠(𝑖) ≤ 200)
<br> Line 𝑁 + 2: ghi 𝑐(1), 𝑐(2), … , 𝑐(𝑀) (1 ≤ 𝑐(𝑖) ≤ 300)


In [1]:
INPUT_FILE_PATH = './data/input2.txt'
MAX_VALUE = 99999999

N, M = 0, 0
t, g, s, c = [], [], [], []

def read_input(file_name):
    global N, M, t, g, s, c

    with open(file_name) as file:
        lines = [line.rstrip() for line in file]

    N, M = tuple(int(x) for x in lines[0].split())

    for i in range(N):
        num_periods, teacher, num_pupils = tuple(int(x) for x in lines[i+1].split())
        t.append(num_periods)
        g.append(teacher)
        s.append(num_pupils)

    c = [int(x) for x in lines[-1].split()]

    if len(t) != N or len(g) != N or len(s) != N or len(c) != M:
        raise Exception('Error when reading data!')

read_input(INPUT_FILE_PATH)

In [2]:
N, M, t, g, s, c

(10,
 2,
 [4, 4, 4, 2, 4, 3, 2, 3, 4, 3],
 [1, 1, 1, 2, 2, 1, 2, 2, 1, 1],
 [15, 18, 15, 18, 11, 15, 27, 18, 13, 10],
 [20, 20])

# MODEL

In [3]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

## VARIABLES & DOMAINS
<br> Với ***i*** là các giá trị chỉ số của lớp thuộc *{0, ..., N-1}*:

- Biến ***class_is_chosen[i]***: Lớp có được xếp hay không
<br> Biến bằng 1 nếu được xếp, bằng 0 nếu bỏ
<br> *D(class_is_chosen[i]) = {0, 1}*

- Biến ***class_half_day[i]***: Buổi học
<br> Từ thứ Hai đến thứ Sáu, mỗi ngày 2 buổi (sáng và chiều), tổng cộng 10 buổi
<br> Biến lưu giá trị chỉ số buổi (một trong 10 buổi)
<br> *D(class_half_day[i]) = {0, ..., 9}*

- Biến ***class_starting[i]***: Tiết học bắt đầu (trong buổi)
<br> Mỗi buổi học sẽ gồm có 6 tiết
<br> Biến lưu giá trị chỉ số tiết (một trong 6 tiết)
<br> *D(class_starting[i]) = {0, ..., 5}*

- Biến ***class_room[i]***: Phòng học
<br> Biến lưu giá trị chỉ số phòng (một trong *M* phòng), hoặc bằng -1 nếu không được xếp phòng
<br> *D(class_room[i]) = {-1, 0, ..., M-1}*

- Biến ***class_room_onehot[i]***: Phòng học dạng vector one-hot
<br> Các tập (*t, g, s*) ta dùng *i* làm chỉ số mảng, tập c ta lấy giá trị như sau: *c[ class_room[i] ]*
<br> Vì model không cho dùng biến làm chỉ số mảng, nên dùng biến này để lấy dữ liệu từ tập c
<br> Biến sẽ là một vector one-hot, kích thước M, giá trị thứ *j* bằng *1* nếu xếp phòng *j*, bằng *0* nếu bỏ
<br> *D(class_room_onehot[i,j]) = {0, 1}*

In [4]:
#########################
## VARIABLES & DOMAINS ##
#########################

class_is_chosen = [] # Lớp có được xếp hay không
class_half_day = [] # Buổi học
class_starting = [] # Tiết học bắt đầu (trong buổi)
class_room = [] # Phòng học
class_room_onehot = [] # Phòng học dạng vector one-hot

for i in range(N):
    class_is_chosen.append(solver.BoolVar(f'class_is_chosen[{i}]'))
    class_half_day.append(solver.IntVar(0, 9, f'class_half_day[{i}]'))
    class_starting.append(solver.IntVar(0, 5, f'class_starting[{i}]'))

    class_room.append(solver.IntVar(-1, M-1, f'class_room[{i}]'))
    class_room_onehot.append([
        solver.BoolVar(name=f'class_room_onehot[{i},{j}]')
        for j in range(M)
    ])

## CONSTRAINTS 1
<br> Các ràng buộc cho mỗi lớp riêng biệt:

- **Ràng buộc 1**: Mỗi lớp được xếp sẽ chiếm 1 phòng duy nhất
<br> Tổng của vector *class_room_onehot[i]* bằng 1 nếu lớp *i* được xếp, bằng 0 nếu lớp bỏ

- **Ràng buộc 2**: Đồng bộ giữa biến phòng học dạng số và dạng vector one-hot
<br> Nếu lớp *i* được chọn, chỉ số phòng *j* bằng 1 trong *class_room_onehot[i]*, giá trị *class_room[i]* bằng *j*
<br> Nếu lớp *i* bỏ, giá trị *class_room[i]* bằng -1

- **Ràng buộc 3**: Mỗi lớp được xếp có kích thước phòng chứa đủ sĩ số lớp 
<br> Nếu lớp *i* được chọn, kích thước phòng được xếp lớn hơn hoặc bằng sĩ số lớp

- **Ràng buộc 4**: Lớp học giới hạn trong một buổi
<br> Nếu lớp *i* được chọn, tiết kết thúc không được quá tiết thứ 6 trong buổi

In [5]:
###################
## CONSTRAINTS 1 ##
###################

for i in range(N):
    ## CONSTRAINT ##
    # Mỗi lớp được xếp sẽ chiếm 1 phòng duy nhất
    solver.Add(
        sum(class_room_onehot[i]) == class_is_chosen[i]
    )

    ## CONSTRAINT ##
    # Đồng bộ giữa biến phòng học dạng số và dạng vector one-hot
    solver.Add(
        sum([j * class_room_onehot[i][j] for j in range(M)]) - (1 - class_is_chosen[i]) == class_room[i]
    )

    ## CONSTRAINT ##
    # Mỗi lớp được xếp có kích thước phòng chứa đủ sĩ số lớp 
    solver.Add(
        sum(c[j] * class_room_onehot[i][j] for j in range(M)) >= class_is_chosen[i] * s[i]
    )

    ## CONSTRAINT ##
    # Lớp học giới hạn trong một buổi
    solver.Add(
        class_starting[i] + t[i] * class_is_chosen[i] <= 6
    )

## CONSTRAINTS 2

<br> Các ràng buộc cho mối quan hệ 2 lớp với nhau
<br> Xét với lớp *i* và lớp *j* khác nhau:

<br> Biến *is_i_not_overlap_j*: coi như được xếp cùng một buổi, lớp *i* và lớp *j* có không chồng giờ nhau hay không
<br> *is_i_not_overlap_j* = (*i* kết thúc trước *j* bắt đầu) *OR* (*j* kết thúc trước *i* bắt đầu)

<br> Biến *is_i_diff_half_day_j*: lớp *i* và lớp *j* có khác buổi nhau hay không
<br> *is_i_diff_half_day_j* = (buổi của *i*) $\ne$ (buổi của *j*)

<br> Biến *is_i_not_overlap_j_truly*: lớp *i* và lớp *j* có khác buổi hoặc cùng buổi nhưng không chồng giờ nhau hay không
<br> *is_i_not_overlap_j_truly* = *is_i_diff_half_day_j* *OR* *is_i_not_overlap_j*

<br> Biến *is_i_and_j_chosen*: lớp *i* và lớp *j* có cùng được xếp lịch hay không
<br> *is_i_and_j_chosen* = (lớp *i* được xếp) *AND* (lớp *j* được xếp)

<br> Biến *is_i_same_room_j*: lớp *i* và lớp *j* có chung phòng hay không
<br> *is_i_same_room_j* = (phòng của i) *bằng* (phòng của j)

- **Ràng buộc 5**: Nếu lớp được xếp và chung giáo viên, các lớp không được xếp chồng giờ nhau
<br> *if* *is_i_and_j_chosen* *AND* (*i* và *j* chung giáo viên): *is_i_not_overlap_j_truly* = 1

- **Ràng buộc 6**: Nếu lớp được xếp và xếp chung phòng, các lớp không được xếp chồng giờ nhau
<br> *if* *is_i_and_j_chosen* *AND* *is_i_same_room_j*: *is_i_not_overlap_j_truly* = 1

In [6]:
###################
## CONSTRAINTS 2 ##
###################

for i in range(N):
    for j in range(N):
        if i == j:
            continue


        # Biến is_i_not_overlap_j: lớp i không bị chồng giờ với lớp j hay không (coi như chung một buổi), D = {0, 1}
        # Biến thể Công thức Bằng
        is_i_not_overlap_j = solver.BoolVar(f'is_i_not_overlap_j[{i},{j}]')
        is_i_before_j = solver.BoolVar(f'is_i_before_j[{i},{j}]')
        is_i_after_j = solver.BoolVar(f'is_i_after_j[{i},{j}]')
        solver.Add((is_i_before_j - 1) * MAX_VALUE <= class_starting[j] - class_starting[i] - t[i])
        solver.Add(class_starting[j] - class_starting[i] - t[i] <= MAX_VALUE * is_i_before_j - 1)
        solver.Add((is_i_after_j - 1) * MAX_VALUE <= class_starting[i] - class_starting[j] - t[j])
        solver.Add(class_starting[i] - class_starting[j] - t[j] <= MAX_VALUE * is_i_after_j - 1)
        solver.Add(is_i_not_overlap_j >= is_i_before_j)
        solver.Add(is_i_not_overlap_j >= is_i_after_j)
        solver.Add(is_i_not_overlap_j <= is_i_before_j + is_i_after_j)

        # Biến is_i_diff_half_day_j: lớp i và j có khác buổi hay không, D = {0, 1}
        # Công thức Không Bằng 
        is_i_diff_half_day_j = solver.BoolVar(f'is_i_diff_half_day_j[{i},{j}]')
        is_i_half_day_before_j = solver.BoolVar(f'is_i_half_day_before_j[{i},{j}]')
        is_i_half_day_after_j = solver.BoolVar(f'is_i_half_day_after_j[{i},{j}]')
        solver.Add((is_i_half_day_before_j - 1) * MAX_VALUE <= class_half_day[j] - class_half_day[i] - 1)
        solver.Add(class_half_day[j] - class_half_day[i] - 1 <= is_i_half_day_before_j * MAX_VALUE - 1)
        solver.Add((is_i_half_day_after_j - 1) * MAX_VALUE <= class_half_day[i] - class_half_day[j] - 1)
        solver.Add(class_half_day[i] - class_half_day[j] - 1 <= is_i_half_day_after_j * MAX_VALUE - 1)
        solver.Add(is_i_diff_half_day_j >= is_i_half_day_before_j)
        solver.Add(is_i_diff_half_day_j >= is_i_half_day_after_j)
        solver.Add(is_i_diff_half_day_j <= is_i_half_day_before_j + is_i_half_day_after_j)

        # Biến is_i_not_overlap_j_truly: lớp i không bị chồng giờ với lớp j hay không (có xét đến khác buổi), D = {0, 1}
        # is_i_not_overlap_j_truly = is_i_diff_half_day_j OR is_i_not_overlap_j (Công thức OR)
        is_i_not_overlap_j_truly = solver.BoolVar(f'is_i_not_overlap_j_truly[{i},{j}]')
        solver.Add(is_i_not_overlap_j_truly >= is_i_diff_half_day_j)
        solver.Add(is_i_not_overlap_j_truly >= is_i_not_overlap_j)
        solver.Add(is_i_not_overlap_j_truly <= is_i_diff_half_day_j + is_i_not_overlap_j)


        # Biến is_i_and_j_chosen: Cả lớp i và lớp j cùng được xếp hay không, D = {0, 1}
        # Công thức AND
        is_i_and_j_chosen = solver.BoolVar(f'is_i_and_j_chosen[{i},{j}]')
        solver.Add(is_i_and_j_chosen <= class_is_chosen[i])
        solver.Add(is_i_and_j_chosen <= class_is_chosen[j])
        solver.Add(is_i_and_j_chosen >= class_is_chosen[i] + class_is_chosen[j] - 1)

        # Biến is_i_same_room_j: Nếu lớp i và lớp j chung phòng
        # Công thức Bằng
        is_i_same_room_j = solver.BoolVar(f'is_i_same_room_j[{i},{j}]')
        is_i_room_idx_before_j = solver.BoolVar(f'is_i_room_idx_before_j[{i},{j}]')
        is_i_room_idx_after_j = solver.BoolVar(f'is_i_room_idx_after_j[{i},{j}]')
        solver.Add((is_i_room_idx_before_j - 1) * MAX_VALUE <= class_room[j] - class_room[i] - 1)
        solver.Add(class_room[j] - class_room[i] - 1 <= is_i_room_idx_before_j * MAX_VALUE - 1)
        solver.Add((is_i_room_idx_after_j - 1) * MAX_VALUE <= class_room[i] - class_room[j] - 1)
        solver.Add(class_room[i] - class_room[j] - 1 <= is_i_room_idx_after_j * MAX_VALUE - 1)
        solver.Add(1 - is_i_same_room_j >= is_i_room_idx_before_j)
        solver.Add(1 - is_i_same_room_j >= is_i_room_idx_after_j)
        solver.Add(1 - is_i_same_room_j <= is_i_room_idx_before_j + is_i_room_idx_after_j)

        # Biến is_i_same_room_j_and_chosen: Nếu lớp i và lớp j cùng được xếp và xếp chung phòng
        # is_i_same_room_j_and_chosen = is_i_same_room_j AND is_i_and_j_chosen (Công thức AND)
        is_i_same_room_j_and_chosen = solver.BoolVar(f'is_i_same_room_j_and_chosen[{i},{j}]')
        solver.Add(is_i_same_room_j_and_chosen <= is_i_same_room_j)
        solver.Add(is_i_same_room_j_and_chosen <= is_i_and_j_chosen)
        solver.Add(is_i_same_room_j_and_chosen >= is_i_same_room_j + is_i_and_j_chosen - 1)


        ## CONSTRAINT ##
        # Nếu lớp được xếp và chung giáo viên, các lớp không được xếp chồng giờ nhau
        if g[i] == g[j]:
            solver.Add(is_i_not_overlap_j_truly >= is_i_and_j_chosen)
        

        ## CONSTRAINT ##
        # Nếu lớp được xếp và xếp chung phòng, các lớp không được xếp chồng giờ nhau
        solver.Add(is_i_not_overlap_j_truly >= is_i_same_room_j_and_chosen)


print('Number of constraints:', solver.NumConstraints())

Number of constraints: 2872


# OBJECTIVES
- Tối đa số lượng lớp được chọn
<br> Tổng *class_is_chosen[i]* với *i* thuộc *{1, ..., N}* là lớn nhất

In [7]:
################
## OBJECTIVES ##
################

# Tối đa số lượng lớp được chọn
solver.Maximize(sum(class_is_chosen))

# OUTPUT

<br> Line 1: contains a positive integer 𝑄
<br> Line 𝑞 + 1 (𝑞 = 1, 2, . . . ,𝑄): contains 3 positive integers 𝑖, 𝑢, and 𝑣 in which class i is assigned to slot u and room u


In [8]:
status = solver.Solve()
solution = {}

if status == pywraplp.Solver.OPTIMAL:
    for idx, val in enumerate(class_is_chosen):
        if val.solution_value():
            solution[idx + 1] = {
                'starting': int(
                    class_half_day[idx].solution_value() * 6 + class_starting[idx].solution_value() + 1
                ),
                'room': int(class_room[idx].solution_value() + 1),
            }
            
else:
    print('No solution found.')

In [9]:
print(len(solution))
for i in solution:
    starting = solution[i]['starting']
    room = solution[i]['room']
    print(f'{i} {starting} {room}')

9
1 3 1
2 9 1
3 15 1
4 13 1
5 3 2
6 28 1
8 19 2
9 33 1
10 22 1


# Các phương pháp mô hình hóa về dạng tuyến tính

Mô hình hóa toán tử ***and***, ***or*** và ***not***:
<br> x, y và z là giá trị nhị phân {0, 1}

<br> (CT AND) với: z = x *and* y
<br> tương đương:
<br>&emsp; z $\le$ x
<br>&emsp; z $\le$ y
<br>&emsp; z $\ge$ x + y - 1

<br> (CT OR) với: z = x *or* y
<br> tương đương:
<br>&emsp; z $\ge$ x
<br>&emsp; z $\ge$ y
<br>&emsp; z $\le$ x + y

<br> (CT NOT) với: z = *not* x
<br> tương đương: z = 1 - x

Mô hình hóa ***if***:
<br>MAX là một số rất lớn, A và B là giá trị nguyên, x là giá trị nhị phân {0, 1}

<br>(cái này chưa dùng tới) với: 
<br>&emsp; *if* A = 0: x = 0
<br>&emsp; *else* A > 0: x = 1
<br>tương đương: 
<br>&emsp; (1/MAX) * x $\le$ A $\le$ MAX * x

<br>(CT Không Âm) với:
<br>&emsp; *if* A $\ge$ 0: x = 1
<br>&emsp; *else* A < 0: x = 0
<br>tương đương: 
<br>&emsp; (x - 1) * MAX $\le$ A $\le$ MAX * x - 1
<br>(nếu muốn A dương thì A-1 không âm)

<br>(CT Bằng) với:
<br>&emsp; *if* A = B: x = 1 else x = 0
<br>tương đương: 
<br>&emsp; temp1 = (A > B) = (A - B - 1 $\ge$ 0) &emsp;(CT Không Âm)
<br>&emsp; temp2 = (B > A) = (B - A - 1 $\ge$ 0) &emsp;(CT Không Âm)
<br>&emsp; x = *not* (temp1 *or* temp2) &emsp;(CT NOT và CT OR)
<br>(bỏ *not* với trường hợp A $\ne$ B)